In [57]:
# importing packages
import pandas as pd
import numpy as np
import altair as alt
import statsmodels.formula.api as smf

python(10930) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [34]:
# Loading pricing data
prices_df = pd.read_csv('https://eco-prices-scrapes.s3.eu-west-2.amazonaws.com/teaching/davies_price_data/db_prices.csv')
items_df = pd.read_csv('https://eco-prices-scrapes.s3.eu-west-2.amazonaws.com/teaching/davies_price_data/db_item_clean.csv')

In [35]:
# Identifying target descriptions
targets = ['HOME CARE ASSISTANT HRLY RATE' , 'RESIDENTIAL HOME' , 'NURSING HOME']

In [36]:
# Getting the target items with codes
target_items = items_df[items_df['description'].isin(targets)]

In [37]:
# Getting the pricing data for target items
care_prices_df = prices_df[prices_df['item_id'].isin(target_items['item_id'])]

In [38]:
# Delete pricing data that takes up too much memory
del prices_df

In [47]:
# Merge in items data frame for descriptions along with prices
care_prices = care_prices_df.merge(items_df , how='left' , on='item_id')

In [48]:
 # convert quote_date to datetime, format is yearmonth
care_prices['quote_date'] = pd.to_datetime(care_prices['quote_date'], format='%Y%m')

In [49]:
# Aggregate to the median prices by month for each target item
care_prices = care_prices.groupby(['description','quote_date']).agg({'price': 'median'}).reset_index()

In [50]:
#Subset data so that the first observation for eeach description type starts on the same date
care_prices = care_prices[care_prices['quote_date'] >= '2014-09-01']

In [58]:
care_prices['months_since_start'] = (care_prices['quote_date'] - care_prices['quote_date'].min()).dt.days

In [59]:
hc_care_prices = care_prices[care_prices['description'] == 'HOME CARE ASSISTANT HRLY RATE']
nh_care_prices = care_prices[care_prices['description'] == 'NURSING HOME']
rh_care_prices = care_prices[care_prices['description'] == 'RESIDENTIAL HOME']

In [60]:
hc_care_prices_sub = hc_care_prices[hc_care_prices['quote_date'] <= '2022-04-01']
nh_care_prices_sub = nh_care_prices[nh_care_prices['quote_date'] <= '2022-04-01']
rh_care_prices_sub = rh_care_prices[rh_care_prices['quote_date'] <= '2022-04-01']

In [61]:
hc_reg = smf.ols('price ~ months_since_start', data=hc_care_prices_sub).fit()
nh_reg = smf.ols('price ~ months_since_start', data=nh_care_prices_sub).fit()
rh_reg = smf.ols('price ~ months_since_start', data=rh_care_prices_sub).fit()

In [66]:
hc_care_prices['price_trend'] = hc_reg.predict(hc_care_prices['months_since_start'])
nh_care_prices['price_trend'] = nh_reg.predict(nh_care_prices['months_since_start'])
rh_care_prices['price_trend'] = rh_reg.predict(rh_care_prices['months_since_start'])

/var/folders/qz/pj0lh7817m3c9ydwgfrtbtf00000gn/T/ipykernel_8093/3326192353.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nh_care_prices['price_trend'] = nh_reg.predict(nh_care_prices['months_since_start'])
/var/folders/qz/pj0lh7817m3c9ydwgfrtbtf00000gn/T/ipykernel_8093/3326192353.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rh_care_prices['price_trend'] = rh_reg.predict(rh_care_prices['months_since_start'])


In [70]:
care_prices_rejoin = pd.concat([hc_care_prices , nh_care_prices , rh_care_prices])

In [72]:
care_prices_rejoin.to_csv('/Users/sambickel-barlow/Desktop/PP434/Portfolio/CC8/care_prices.csv')